In [2]:
from enums import CollectionNames, EmbeddingModels
from qdrant_client.models import Filter, FieldCondition, MatchValue

from retrieval_util import (
    get_qdrant_client,
    get_embeddings_function,
    retrieve_run_dict_with_documents_with_scores,
    create_llm_for_self_query_retriever
)

import os
import json
import logging
from ranx import Qrels, Run, evaluate, compare

os.environ[
    "TOKENIZERS_PARALLELISM"] = "false"  # Verhindert Fehler beim Erzeugen von Embeddings (Tritt sporadisch auf, konnte ich nicht gezielt reproduzieren)
logging.basicConfig(level=logging.INFO)

# retrieval_util.py SelfQueryRetriever use_original_query=True bzw. False gesetzt?
-> nach Änderung den Jupyter-Server neu starten, um neue Versionen der Dateien zu laden

In [3]:
# Konstanten
MAX_DOCUMENTS = 10
BASE_PATH = "./infloat_retrieval_results_AT_10"
LLM = create_llm_for_self_query_retriever()
FRAGEN = {
    1: "Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben?",
    2: "Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest?",
    3: "Welche Fachgebiete sind an der FH vertreten?",
    4: "Ist eine Beurlaubung während des Studiums möglich?",
    5: "Welche Studienmodelle werden angeboten?",
    6: "Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben?",
    7: "Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut?",
    8: "Was muss ich beachten, wenn ich bei einer Prüfung krank bin?",
    9: "Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen?",
    10: "Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen?"
}
METRICS = ["ndcg@10", "mrr@10", "map@10", "precision@10", "recall@10", "hits@10", "hit_rate@10"]

In [4]:
# Alle relevanten Filter für die Fragen
def create_field_condition(field, value):
    return FieldCondition(
        key=f"metadata.{field}",
        match=MatchValue(value=value)
    )


FILTERS = {
    1: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Meschede"),
        create_field_condition("abschluss", "Master")
    ]),
    2: Filter(must=[
        create_field_condition("standort", "Soest"),
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    3: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    4: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    5: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    6: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    7: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Soest"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    8: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    9: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    10: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
}

In [5]:
def save_results_to_file(file_path, results_dict):
    try:
        with open(file_path, "w") as file:
            json.dump(results_dict, file, indent=4)
        print(f"Ergebnisse wurden erfolgreich in {file_path} gespeichert.")
    except Exception as e:
        print(f"Fehler beim Speichern der Datei: {e}")

In [6]:
token_based_qrels_dict = {
    "q_1": {
        "d_1021": 10,
        "d_1022": 1
    },
    "q_2": {
        "d_1301": 10
    },
    "q_3": {
        "d_1476": 10,
        "d_1480": 10,
        "d_1481": 2
    },
    "q_4": {
        "d_1498": 10,
        "d_1499": 10,
        "d_1500": 2
    },
    "q_5": {
        "d_1482": 10,
        "d_1483": 10,
        "d_1484": 10,
        "d_1485": 10
    },
    "q_6": {
        "d_18": 10
    },
    "q_7": {
        "d_1192": 10,
        "d_1193": 8
    },
    "q_8": {
        "d_1530": 10,
        "d_1553": 10,
        "d_1554": 10,
        "d_1555": 10
    },
    "q_9": {
        "d_704": 10
    },
    "q_10": {
        "d_14": 10,
        "d_15": 10
    }
}

TOKEN_BASED_QRELS = Qrels(token_based_qrels_dict)

In [7]:
recursive_qrels_dict = {
    "q_1": {
        "d_1371": 10,
        "d_1370": 1
    },
    "q_2": {
        "d_1866": 10
    },
    "q_3": {
        "d_2292": 10,
        "d_2293": 2,
        "d_2286": 10
    },
    "q_4": {
        "d_2313": 10,
        "d_2314": 10,
        "d_2315": 2
    },
    "q_5": {
        "d_2290": 4,
        "d_2295": 10,
        "d_2296": 10,
        "d_2297": 10,
        "d_2298": 10
    },
    "q_6": {
        "d_25": 10
    },
    "q_7": {
        "d_1586": 10
    },
    "q_8": {
        "d_2388": 10,
        "d_2389": 10,
        "d_2390": 10,
        "d_2356": 8,
        "d_2357": 8
    },
    "q_9": {
        "d_946": 10
    },
    "q_10": {
        "d_19": 10
    }
}

RECURSIVE_QRELS = Qrels(recursive_qrels_dict)

In [8]:
embeddings = get_embeddings_function(model_name=EmbeddingModels.INFLOAT.value)

/Users/bastian/Desktop/BachelorProjektAbgabe/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-16 12:55:39,499 [INFO] PyTorch version 2.6.0 available.
2025-03-16 12:55:39,737 [INFO] Use pytorch device_name: mps
2025-03-16 12:55:39,738 [INFO] Load pretrained SentenceTransformer: intfloat/multilingual-e5-large


In [9]:
qdrant_url = "http://localhost:6333"
token_based_client = get_qdrant_client(embeddings=embeddings, collection_name=CollectionNames.INFLOAT_TOKEN_BASED.value,
                                       qdrant_url=qdrant_url)
recursive_client = get_qdrant_client(embeddings=embeddings, collection_name=CollectionNames.INFLOAT_RECURSIVE.value,
                                     qdrant_url=qdrant_url)

2025-03-16 12:55:44,964 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-03-16 12:55:44,972 [INFO] HTTP Request: GET http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks "HTTP/1.1 200 OK"
2025-03-16 12:55:45,291 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-03-16 12:55:45,298 [INFO] HTTP Request: GET http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks "HTTP/1.1 200 OK"


# Token-Basierte Collections Retrieval Evaluation

In [10]:
# Run dicts ohne Filter, mit optimalen Filtern und mit Self-Query-Retriever erzeugen
token_based_result_no_filters = retrieve_run_dict_with_documents_with_scores(client=token_based_client,
                                                                             fragen_dict=FRAGEN, mode="no_filters",
                                                                             max_documents=MAX_DOCUMENTS)

token_based_result_optimal_filters = retrieve_run_dict_with_documents_with_scores(client=token_based_client,
                                                                                  fragen_dict=FRAGEN,
                                                                                  mode="optimal_filters",
                                                                                  filter_dict=FILTERS,
                                                                                  max_documents=MAX_DOCUMENTS)

token_based_result_self_query_retriever = retrieve_run_dict_with_documents_with_scores(client=token_based_client,
                                                                                       fragen_dict=FRAGEN,
                                                                                       mode="self_query_retriever",
                                                                                       llm=LLM,
                                                                                       search_kwargs={
                                                                                           "k": MAX_DOCUMENTS})

2025-03-16 12:55:45,465 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 12:55:45,562 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 12:55:45,651 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 12:55:45,690 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 12:55:45,794 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 12:55:45,877 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16

In [11]:
# Run dicts in Dateien schreiben, um sie einsehen zu können
save_results_to_file(file_path=f"{BASE_PATH}/token_based_result_no_filters.json",
                     results_dict=token_based_result_no_filters)
save_results_to_file(file_path=f"{BASE_PATH}/token_based_result_optimal_filters.json",
                     results_dict=token_based_result_optimal_filters)
save_results_to_file(file_path=f"{BASE_PATH}/token_based_result_self_query_retriever.json",
                     results_dict=token_based_result_self_query_retriever)

Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_5/token_based_result_no_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_5/token_based_result_optimal_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_5/token_based_result_self_query_retriever.json gespeichert.


In [12]:
# Runs erzeugen
run_token_based_result_no_filters = Run(token_based_result_no_filters, name="token_based_result_no_filters")
run_token_based_result_optimal_filters = Run(token_based_result_optimal_filters,
                                             name="token_based_result_optimal_filters")
run_token_based_result_self_query_retriever = Run(token_based_result_self_query_retriever,
                                                  name="token_based_result_self_query_retriever")

In [13]:
# Evaluation durchführen
token_based_scores_no_filters = evaluate(qrels=TOKEN_BASED_QRELS, run=run_token_based_result_no_filters,
                                         metrics=METRICS)
token_based_scores_optimal_filters = evaluate(qrels=TOKEN_BASED_QRELS, run=run_token_based_result_optimal_filters,
                                              metrics=METRICS)
token_based_scores_self_query_retriever = evaluate(qrels=TOKEN_BASED_QRELS,
                                                   run=run_token_based_result_self_query_retriever, metrics=METRICS)

In [14]:
# Scores in Dateien schreiben, um sie einsehen zu können
save_results_to_file(file_path=f"{BASE_PATH}/token_based_scores_no_filters.json",
                     results_dict=token_based_scores_no_filters)
save_results_to_file(file_path=f"{BASE_PATH}/token_based_scores_optimal_filters.json",
                     results_dict=token_based_scores_optimal_filters)
save_results_to_file(file_path=f"{BASE_PATH}/token_based_scores_self_query_retriever.json",
                     results_dict=token_based_scores_self_query_retriever)

Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_5/token_based_scores_no_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_5/token_based_scores_optimal_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_5/token_based_scores_self_query_retriever.json gespeichert.


In [15]:
report = compare(
    qrels=TOKEN_BASED_QRELS,
    runs=[run_token_based_result_no_filters, run_token_based_result_optimal_filters,
          run_token_based_result_self_query_retriever],
    metrics=METRICS
)

In [16]:
print(report)

#    Model                                      NDCG@5    MRR@5    MAP@5    P@5    Recall@5    Hits@5    Hit Rate@5
---  ---------------------------------------  --------  -------  -------  -----  ----------  --------  ------------
a    token_based_result_no_filters               0.321    0.433    0.265   0.2        0.3         1             0.5
b    token_based_result_optimal_filters          0.385    0.5      0.341   0.24       0.367       1.2           0.5
c    token_based_result_self_query_retriever     0.321    0.433    0.265   0.2        0.3         1             0.5


In [17]:
with open(f"{BASE_PATH}/token_report@{MAX_DOCUMENTS}.txt", "w") as file:
    file.write(str(report))

# Recursive-Collections Retrieval Evaluation

In [18]:
recursive_result_no_filters = retrieve_run_dict_with_documents_with_scores(client=recursive_client, fragen_dict=FRAGEN,
                                                                           mode="no_filters",
                                                                           max_documents=MAX_DOCUMENTS)
recursive_result_optimal_filters = retrieve_run_dict_with_documents_with_scores(client=recursive_client,
                                                                                fragen_dict=FRAGEN,
                                                                                mode="optimal_filters",
                                                                                filter_dict=FILTERS,
                                                                                max_documents=MAX_DOCUMENTS)
recursive_result_self_query_retriever = retrieve_run_dict_with_documents_with_scores(client=recursive_client,
                                                                                     fragen_dict=FRAGEN,
                                                                                     mode="self_query_retriever",
                                                                                     llm=LLM,
                                                                                     search_kwargs={"k": MAX_DOCUMENTS})

2025-03-16 12:57:12,891 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 12:57:12,937 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 12:57:12,975 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 12:57:13,015 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 12:57:13,080 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 12:57:13,129 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 12:57:13,17

In [19]:
save_results_to_file(file_path=f"{BASE_PATH}/recursive_result_no_filters.json",
                     results_dict=recursive_result_no_filters)
save_results_to_file(file_path=f"{BASE_PATH}/recursive_result_optimal_filters.json",
                     results_dict=recursive_result_optimal_filters)
save_results_to_file(file_path=f"{BASE_PATH}/recursive_result_self_query_retriever.json",
                     results_dict=recursive_result_self_query_retriever)

Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_5/recursive_result_no_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_5/recursive_result_optimal_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_5/recursive_result_self_query_retriever.json gespeichert.


In [20]:
run_recursive_result_no_filters = Run(recursive_result_no_filters, name="recursive_result_no_filters")
run_recursive_result_optimal_filters = Run(recursive_result_optimal_filters, name="recursive_result_optimal_filters")
run_recursive_result_self_query_retriever = Run(recursive_result_self_query_retriever,
                                                name="recursive_result_self_query_retriever")

In [21]:
recursive_scores_no_filters = evaluate(qrels=RECURSIVE_QRELS, run=run_recursive_result_no_filters, metrics=METRICS)
recursive_scores_optimal_filters = evaluate(qrels=RECURSIVE_QRELS, run=run_recursive_result_optimal_filters,
                                            metrics=METRICS)
recursive_scores_self_query_retriever = evaluate(qrels=RECURSIVE_QRELS, run=run_recursive_result_self_query_retriever,
                                                 metrics=METRICS)

In [22]:
save_results_to_file(file_path=f"{BASE_PATH}/recursive_scores_no_filters.json",
                     results_dict=recursive_scores_no_filters)
save_results_to_file(file_path=f"{BASE_PATH}/recursive_scores_optimal_filters.json",
                     results_dict=recursive_scores_optimal_filters)
save_results_to_file(file_path=f"{BASE_PATH}/recursive_scores_self_query_retriever.json",
                     results_dict=recursive_scores_self_query_retriever)

Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_5/recursive_scores_no_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_5/recursive_scores_optimal_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_5/recursive_scores_self_query_retriever.json gespeichert.


In [23]:
report = compare(
    qrels=RECURSIVE_QRELS,
    runs=[run_recursive_result_no_filters, run_recursive_result_optimal_filters,
          run_recursive_result_self_query_retriever],
    metrics=METRICS
)

In [24]:
print(report)

#    Model                                    NDCG@5    MRR@5    MAP@5    P@5    Recall@5    Hits@5    Hit Rate@5
---  -------------------------------------  --------  -------  -------  -----  ----------  --------  ------------
a    recursive_result_no_filters               0.404    0.467    0.377   0.3        0.507       1.5           0.6
b    recursive_result_optimal_filters          0.447    0.533    0.44    0.32       0.54        1.6           0.6
c    recursive_result_self_query_retriever     0.404    0.467    0.377   0.3        0.507       1.5           0.6


In [25]:
with open(f"{BASE_PATH}/recursive_report@{MAX_DOCUMENTS}.txt", "w") as file:
    file.write(str(report))

Anzumerken ist, dass bei der Nutzung des SelfQueryRetriever mit dem llama3.1:8b Modell für einige Fragen keine Filter erzeugt werden können, weil die Antwort des LLMs nicht richtig formatiert ist.<br>
Zeitgleich werden die Filter für die in der Ausarbeitung genutzten 10 Fragen immer gleich erzeugt.<br>


Bei der Prüfung der abgerufenen Dokumente für den SelfQueryRetriever stimmen diese im durchlauf aus dem Ordner 4-Retrieval-LnagChain-Qdrant mit den Ergebnissen aus Ordner 6-Evaluation-Retrieval überein.<br>
Somit verhält er sich voraussichtlich immer gleich bei gleichen Fragen mit dem gleichen LLM für die Erzeugung der Filter.<br>

Laut LangChain-Doku sollen die Probleme mit leistungsstärkeren Modellen nicht auftreten.<br>


In der Datenbasis sind einige Dokumente, die ausschließlich den Standort als Filterkriterium haben.
Diese wurden nicht weiter mit den Fragen abgedeckt, da der Umfang der Chunks dort zu klein ist, sodass immer alle relevanten Informationen abgerüfen würden, wenn der Filter optimal gesetzt ist.
Um die Auswertungen der Abfragen nicht zu verfälschen und immer 10 Dokumente abrufen zu können, entfallen Fragen mit bezug auf diese Daten. Sie beziehen sich ausschließlich auf Informationen der Webseite zu den spezifischen Standorten.

Dies ist erst in der Evaluation klar geworden. Um die Chunks nicht alle neu zu erzeugen und damit ggf. andere IDs bei den abrufen zu erhalten, wurden ein Ausschluss der Fragen zu diesen Daten bevorzugt.